In [72]:
import time
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import os
from urllib.parse import urlparse, parse_qs

In [84]:
product_df =pd.read_csv("./data/fathur_new_data.csv")
hiking_df = product_df[product_df['hiking']==1]



In [85]:
cosplay_df = product_df[product_df['hiking']==1]

In [86]:
renter_df = pd.concat([hiking_df['renters'], cosplay_df['renters']])
renter_df = renter_df.drop_duplicates()
renter_df = renter_df.dropna()
renter_df = renter_df.reset_index(drop=True)
renter_df = renter_df.head(1000)
renter_df

0           Injection Project
1            Java Seven Store
2                mytactical_1
3                  naondstore
4                     TWIC_id
                ...          
995            Toko Priadybam
996           Parama Konveksi
997    mayangsaridewitaoshops
998            abdullahgalang
999                chad store
Name: renters, Length: 1000, dtype: object

In [87]:
import time
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs
import logging

In [88]:
def replace_spaces(string):
    if ' ' in string:
        return string.replace(' ', '%20')
    else:
        return string

In [89]:
def get_renters_info(df):
    infos = []
    names = df
    
    # Initialize the Chrome driver
    driver = webdriver.Chrome()
    
    visited_urls = set()

    # Loop and parse each link
    for name in names:

        if len(infos) >= 100:
            break
            
        shop_query = replace_spaces(name)
        search = f'https://www.tokopedia.com/search?st=shop&q={shop_query}'
        driver.get(search)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        try:
            link = soup.find('a', class_='css-1tuarat')['href']
            # Parse the URL and get the direct URL
            parsed_url = urlparse(str(link))
            query_params = parse_qs(parsed_url.query)
            direct_url = query_params.get('r', [link])[0]
            
            # Visit the direct URL if not already visited
            if direct_url not in visited_urls:
                driver.get(direct_url)
                time.sleep(5)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                for _ in range(25):
                    time.sleep(0.5)
                    driver.execute_script("window.scrollBy(0,250)")
                
                # Extract the necessary information
                renter_name = soup.find('h1', class_='css-fzzhh3')
                renter_image = soup.find('img', class_='css-46cmny eit69wn2')
                
                main_container = soup.find('div', class_='css-wifapm')
                first_inner_container = main_container.find('div', class_='css-14zidxb', attrs={'data-separator': 'true'})
                first_content = first_inner_container.find('div', class_='css-9pw5ea')
                
                description = first_content.find_all('p')[1].find('span').text

                second_inner_container = main_container.find('div', id='shopseo')
    
                # Append the extracted information to the list
                infos.append({
                    'renter_name': renter_name.text if renter_name else 'N/A',
                    'renter_image': renter_image['src'] if renter_image else 'N/A',
                    'description': description if description else 'N/A',
                    
                })
                time.sleep(1)
                
                # Mark the URL as visited
                visited_urls.add(direct_url)

            else:
                continue
                
        except Exception as e:
            continue

    # Quit the driver, then return the DataFrame
    driver.quit()
    return pd.DataFrame(infos)

In [90]:
renters_info = get_renters_info(renter_df)
renters_info

,renter_name,renter_image,description
0,mytactical_1,https://images.tokopedia.net/img/cache/215-squ...,mengutamakan kwalitas
1,naondstore,https://images.tokopedia.net/img/cache/215-squ...,Selamat datang di online shop naond store! Men...
2,TWIC_id,https://images.tokopedia.net/img/cache/215-squ...,"selamat datang di TWIC-id , \ntoko buka jam 09..."
3,syahr tactical,https://images.tokopedia.net/img/cache/215-squ...,Madre.qu Jakarta merupakan cabang resmi dari M...
4,Master tactical.id,https://images.tokopedia.net/img/cache/215-squ...,KAMI KONVEKSI TERBESAR SIAP MELAYANI ANDA BAIK...
...,...,...,...
95,solotactical,https://images.tokopedia.net/img/cache/215-squ...,#SMS/WA : 081329999010
96,nomi2,https://images.tokopedia.net/img/cache/215-squ...,Retail & Grosir dengan harga yang memuaskan pe...
97,suhuks store id,https://images.tokopedia.net/img/cache/215-squ...,"Hi, guys! Silahkan tanya dulu yah kalau ada ba..."
98,Quinn Shop Indonesia,https://images.tokopedia.net/img/cache/215-squ...,Menjual segala macam keperluan Baju anak remaj...


In [99]:
renters_info = renters_info.drop_duplicates()
renters_info = renters_info.dropna()
renters_info = renters_info.reset_index(drop=True)
renters_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   renter_name   100 non-null    object
 1   renter_image  100 non-null    object
 2   description   100 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


In [102]:
renters_info.to_csv('./data/renters_info.csv', index=False)